In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.base import data_path
from api.data.curves import curves

In [3]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Settings

In [37]:
cds_name = "TURKEY CDS USD SR 5Y D14 Curncy"
trade_date = pd.Timestamp('2021-06-02') # pd.Timestamp.today().floor('d')
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve'
date_start = pd.Timestamp('2021-06-02')
date_end = pd.Timestamp('2022-06-23')

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [5]:
### Bloomberg Connection
bq = BlpQuery(timeout=50000).start()

#### Local data

In [32]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

In [ ]:
DataGrid(cds_ref)

#### Functions

In [33]:
def get_cds_specs(cds_name):
    return cds_ref.query(f"name == '{cds_name}'").squeeze()


def get_cds_spreads(cds_name):
    return cds_hist.query(f"name == '{cds_name}'").set_index('date').dropna()


def get_risk_free_curve(curve_name):
    curve = [curve for curve in curves if curve['name'] == curve_name][0]
    curve_hist_ = curve_hist.query(f'curve_name == "{curve["name"]}"')
    curve_hist_ = curve_hist_.assign(
        tenor=pd.Categorical(
            curve_hist_["tenor"],
            categories=[memb['tenor'] for memb in curve['members']],
            ordered=True
        )
    )
    return curve_hist_


def get_cds_curve(cds_name):
    cds_name_ref = cds_ref.loc[cds_ref.name.str.contains(get_cds_name_root(cds_name))]
    cds_curve_ = cds_hist.loc[cds_hist['security'].isin(cds_name_ref['security'])]
    cds_curve_ = cds_curve_.merge(cds_ref[['security', 'tenor']], on='security')
    cds_curve_['tenor'] = cds_curve_['tenor'].apply(lambda x: get_tenor_des(x))
    return cds_curve_ 
    

def get_cds_name_root(cds_name):
    pattern = re.compile(r'([a-zA-Z,\s]+) \d+.+')
    match = pattern.match(cds_name)
    return match.groups()[0]


def get_tenor_des(tenor: float):
    return f"{tenor:.0f}Y" if tenor > 1 else f"{int(12 * tenor):.0f}M"


def get_ql_date(date):
    return ql.Date(date.day, date.month, date.year)


def get_term_date(date):
    y, m, d = date.year, date.month, date.day
    day = 20
    if date < pd.Timestamp(year=y, month=3, day=20):
        year = y - 1
        month = 12
    elif date < pd.Timestamp(year=y, month=9, day=20):
        year = y
        month = 6
    else:
        year = y
        month = 12
    start = pd.Timestamp(year=year, month=month, day=day)
    return start + pd.tseries.offsets.DateOffset(years=5)


def get_cds_implied_price(cds_name, dates, spreads):
    if isinstance(dates, pd.DatetimeIndex):
        dates = dates.to_series()
    if isinstance(dates, pd.Series):
        dates = dates.dt.strftime(bbg_dt_fmt).to_list()
    if isinstance(spreads, pd.Series):
        spreads = spreads.to_list()
        
    cds_implied_prices = bq.bdph(
        securities=[cds_name],
        fields=["CDS_QUOTED_PRICE"],
        id_override=dates,
        id_override_field="SW_CURVE_DT",
        CDS_FLAT_SPREAD=spreads,
        id_date=True
    )
    cds_implied_prices = cds_implied_prices.rename({'SW_CURVE_DT': 'date', 'CDS_QUOTED_PRICE': 'cds_implied_price'},
                                                   axis=1)
    return cds_implied_prices

In [73]:
def get_cds(
    cds_specs: [dict, pd.Series],
    risk_free_curve: [dict, pd.Series],
    trade_date: pd.Timestamp,
    cds_spread: float = None,
    cds_curve: [dict, pd.Series] = None,
    notional=10000000,
    coupon=100
):
    if not any([cds_spread, cds_curve is not None]):
        raise ValueError('at least one between "cds_spread", or "cds_curve" must be specified')
    if all([cds_spread, cds_curve is not None]):  
        # if both cds_spread and cds_curve are specified, give precedence to cds_spread
        # this will trigger the FlatHazardRate curve which is the standard ISDA model
        cds_curve = None
    
    if isinstance(cds_specs, dict):
        cds_specs = pd.Series(cds_specs)
        
    if cds_spread:
        spread = cds_spread
    else:  # if this point is reached cds_curve  
        if isinstance(cds_curve, dict):
            cds_curve = pd.Series(cds_curve)
        tenor = f"{cds_specs['tenor']:.0f}Y" if cds_specs['tenor'] > 1 else f"{int(12 * cds_specs['tenor']):.0f}M"
        spread = cds_curve[tenor]
    
    currency = cds_specs['currency']
    ql_currency = getattr(ql, f"{currency}Currency")()
    side = ql.Protection.Buyer if notional > 0 else ql.Protection.Seller
    notional = abs(notional)
    recovery = cds_specs['recovery']

    if currency == 'EUR':
        ibor_float_freq = 6
        swap_fixed_freq = ql.Annual
    else:
        ibor_float_freq = 3
        swap_fixed_freq = ql.Semiannual

    deposit_tenors = [
        '1M', 
        '2M', 
        '3M', 
        '6M', 
        '1Y'
    ]
    swap_tenors = [
        '2Y', 
        '3Y', 
        '4Y', 
        '5Y', 
        '6Y',
        '7Y',
        '8Y',
        '9Y',
        '10Y',
        '12Y',
        '15Y',
        '20Y',
        '25Y',
        '30Y'
    ]

    deposit_data = {tenor: risk_free_curve[tenor] for tenor in deposit_tenors if tenor in risk_free_curve}
    swap_data = {tenor: risk_free_curve[tenor] for tenor in swap_tenors if tenor in risk_free_curve}
    # deposit_data = risk_free_curve.loc[risk_free_curve.index.intersection(deposit_tenors)]
    # swap_data = risk_free_curve.loc[risk_free_curve.index.intersection(swap_tenors)]
    
    ql_trade_date = get_ql_date(trade_date)
    ql_term_date = get_ql_date(get_term_date(trade_date))
    ql_upfront_date = ql.WeekendsOnly().advance(ql_trade_date, 3*ql.Period(ql.Daily))
    period_parser = ql.PeriodParser()
    ql.Settings.instance().setEvaluationDate(ql_trade_date)

    deposit_rate_helpers = [
        ql.DepositRateHelper(
            rate / 100,  # rate
            period_parser.parse(tenor),  # tenor                                      
            2,  # fixingDays 
            ql.WeekendsOnly(),
            ql.ModifiedFollowing,
            False, 
            ql.Actual360()
        ) for tenor, rate in deposit_data.items()
    ]

    isda_ibor = ql.IborIndex(
        'IsdaIbor',  # familyName
        ibor_float_freq * ql.Period(ql.Monthly),  # tenor 
        2,  # settlementDays
        ql_currency,  # currency
        ql.WeekendsOnly(),  # fixingCalendar
        ql.ModifiedFollowing, # convention
        False, # endOfMonth
        ql.Actual360(),  # dayCounter
    )

    swap_rate_helpers = [
        ql.SwapRateHelper(
            rate / 100,  # rate
            period_parser.parse(tenor),  # tenor
            ql.WeekendsOnly(),  # calendar
            swap_fixed_freq,  # fixedFrequency
            ql.ModifiedFollowing,  # fixedConvention
            ql.Thirty360(),  # fixedDayCount
            isda_ibor,  # iborIndex
        ) for tenor, rate in swap_data.items()
    ]

    isda_rate_helpers = [*deposit_rate_helpers, *swap_rate_helpers]
    rate_curve = ql.PiecewiseFlatForward(ql_trade_date, isda_rate_helpers, ql.Actual365Fixed())
    discount_curve = ql.YieldTermStructureHandle(rate_curve)

    cds_schedule = ql.Schedule(
        ql_trade_date, 
        ql_term_date,
        3*ql.Period(ql.Monthly),
        ql.WeekendsOnly(),
        ql.Following, 
        ql.Unadjusted,
        ql.DateGeneration.CDS,
        False
    )

    quoted_trade = ql.CreditDefaultSwap(
        side,  # side
        notional,  # nominal
        0,  # upfront
        spread / 10000,  # spread
        cds_schedule,  # Schedule
        ql.Following, 
        ql.Actual360(), 
        True, 
        True, 
        ql_trade_date,
        ql_upfront_date, 
        ql.FaceValueClaim(), 
        ql.Actual360(True),
        True
    )

    if cds_spread:  # assuming FlatHazardRate
        h = quoted_trade.impliedHazardRate(
            0, 
            discount_curve, 
            ql.Actual365Fixed(),
            recovery, 
            1e-10,
            ql.CreditDefaultSwap.ISDA
        )
        pd_curve_ = ql.FlatHazardRate(
            0, 
            ql.WeekendsOnly(),
            ql.QuoteHandle(ql.SimpleQuote(h)),
            ql.Actual365Fixed()
        )
    else:  # assuming cds_curve has been passed as an argument
        cds_helpers = [
            ql.SpreadCdsHelper(
                spread / 10000,  # running_spread
                period_parser.parse(tenor),  # tenor
                0,  # settlementDays ?
                ql.WeekendsOnly(),  # calendar
                ql.Quarterly,  # frequency
                ql.Following,  # paymentConvention
                ql.DateGeneration.CDS,  # rule
                ql.Actual360(),  # dayCounter
                recovery,  # recoveryRate
                discount_curve,  # discountCurve
                True,  # settlesAccrual
                True,  # paysAtDefaultTime
            ) for tenor, spread in cds_curve.items()]
        pd_curve_ = ql.PiecewiseFlatHazardRate(trade_date, cds_helpers, ql.Actual365Fixed())

    pd_curve = ql.DefaultProbabilityTermStructureHandle(pd_curve_)

    engine = ql.IsdaCdsEngine(pd_curve, recovery, discount_curve)

    conventional_trade = ql.CreditDefaultSwap(
        side,  # side
        notional,  # notional
        0,  # upfront
        coupon / 10000,  # spread
        cds_schedule,  # schedule
        ql.Following,  # paymentConvention
        ql.Actual360(),  # dayCounter
        True,  # settlesAccrual
        True,  # paysAtDefaultTime
        ql_trade_date,  # protectionStart
        ql_upfront_date,  # upfrontDate
        ql.FaceValueClaim(),  # Claim
        ql.Actual360(True),  # lastPeriodDayCounter
        True,  # rebatesAccrual
    )
    conventional_trade.setPricingEngine(engine)
    return conventional_trade

#### CDS

In [38]:
cds_specs = get_cds_specs(cds_name)
cds_curve = get_cds_curve(cds_name)
cds_spread  = get_cds_spreads(cds_name).loc[date_start:date_end]
risk_free_curve = get_risk_free_curve(curve_name)
notional = 10000000
coupon=100
flat = True

In [50]:
n =10
cds_ip = get_cds_implied_price(
    cds_name=cds_name,
    dates=cds_spread.index[:n],
    spreads=cds_spread['last_price'][:n]
)

In [71]:
rfc = {               
'1M':0.0888,     
'2M':0.1169,     
'3M':0.1285,     
'6M':0.1749,     
'12M':0.2469,     
'2Y':0.2335,          
'3Y':0.4323,          
'4Y':0.6665,          
'5Y':0.8935,          
'6Y':1.0868,          
'7Y':1.2600,          
'8Y':1.3863,          
'9Y':1.4943,          
'10Y':1.5863,         
'12Y':1.7188,         
'15Y':1.8520,         
'20Y':1.9703,         
'25Y':1.9933,         
'30Y':2.0155,         
}                     

In [74]:
res = {}
for i, (date, row) in enumerate(cds_spread.iterrows()):
    # rfc = risk_free_curve.query(f'date == "{date:{dt_fmt}}"').set_index('tenor')['last_price']
    cds_curve_ = cds_curve.loc[cds_curve['date'] == date].set_index('tenor')['last_price']
    spread = row['last_price']
    
    cds = get_cds(
        cds_specs=cds_specs,
        risk_free_curve=rfc,
        trade_date=date,
        cds_spread=spread,
        notional=notional,
        coupon=coupon
    )
    res[date] = {'npv': cds.NPV(), 'upfront': 100 * (1+cds.fairUpfront()), 'spread': spread}
    
    if n and i + 1 == n:
        break
    
    print(f"{date:%Y-%m-%d} NPV: {cds.NPV()}, Price: {100 * (1+cds.fairUpfront())}, spread: {spread}")
    break

2021-06-02 NPV: 1346058.2827496114, Price: 86.53925116397228, spread: 405.0


In [68]:
get_term_date(date)

Timestamp('2026-06-20 00:00:00')

In [70]:
rfc.sort_index()

tenor
1M     0.0171
2M     0.0260
3M     0.0310
6M     0.0390
1Y     0.0505
2Y     0.1141
3Y     0.2841
4Y     0.5130
5Y     0.7270
6Y     0.9150
7Y     1.0740
8Y     1.2000
9Y     1.3000
10Y    1.3840
12Y    1.5120
15Y    1.6321
20Y    1.7300
25Y    1.7610
30Y    1.7710
Name: last_price, dtype: float64

In [65]:
res = pd.DataFrame(res).T
res

npv    upfront  spread
2021-06-02  1.351116e+06  86.488810   405.0
2021-06-03  1.370491e+06  86.295058   410.0
2021-06-04  1.347953e+06  86.520439   404.5
2021-06-07  1.326073e+06  86.739249   399.5
2021-06-08  1.297412e+06  87.025859   392.5
2021-06-09  1.256528e+06  87.434692   382.5
2021-06-10  1.215469e+06  87.845284   372.5
2021-06-11  1.208802e+06  87.911947   371.0
2021-06-14  1.225322e+06  87.746762   375.5
2021-06-15  1.259088e+06  87.409099   384.0

In [51]:
cds_ip

date                         security  cds_implied_price
0 2021-06-02  TURKEY CDS USD SR 5Y D14 Curncy          86.540016
1 2021-06-03  TURKEY CDS USD SR 5Y D14 Curncy          86.344278
2 2021-06-04  TURKEY CDS USD SR 5Y D14 Curncy          86.576881
3 2021-06-07  TURKEY CDS USD SR 5Y D14 Curncy          86.784350
4 2021-06-08  TURKEY CDS USD SR 5Y D14 Curncy          87.070937
5 2021-06-09  TURKEY CDS USD SR 5Y D14 Curncy          87.473931
6 2021-06-10  TURKEY CDS USD SR 5Y D14 Curncy          87.878430
7 2021-06-11  TURKEY CDS USD SR 5Y D14 Curncy          87.940339
8 2021-06-14  TURKEY CDS USD SR 5Y D14 Curncy          87.781568
9 2021-06-15  TURKEY CDS USD SR 5Y D14 Curncy          87.452016

In [60]:
res['upfront'].rename('prices') - cds_ip.set_index('date')['cds_implied_price'].rename('bbg')

2021-06-02   -0.051205
2021-06-03   -0.049220
2021-06-04   -0.056442
2021-06-07   -0.045101
2021-06-08   -0.045078
2021-06-09   -0.039239
2021-06-10   -0.033146
2021-06-11   -0.028392
2021-06-14   -0.034807
2021-06-15   -0.042917
dtype: float64

In [28]:
100 * (1 + cds.fairUpfront())

90.91812687825173

In [26]:
date

Timestamp('2021-06-02 00:00:00')

In [27]:
cds.NPV()

908185.1552484029

In [ ]:
cds_curve_

In [ ]:
cds_ref[['security', 'tenor']]

In [ ]:
c = cds_curve_.merge(cds_ref[['security', 'tenor']], on='security')
c['tenor'] = c['tenor'].apply(lambda x: get_tenor_des(x))

In [ ]:
c['tenor'] = c['tenor'].apply(lambda x: get_tenor_des(x))

In [ ]:
cds_curve.loc[cds_curve['date'] == date]

In [ ]:

cdss = {}
up = []
for date in cds_hist_.index:
    cds = get_cds(
    cds_name=cds_name,
    curve_name=curve_name,
    trade_date=date
)
    
    # cdss[date] = cds
    up.append({'up': 100 * (1 + cds.fairUpfront()), 'npv': cds.NPV()})

In [ ]:
pd.DataFrame(up, index=cds_hist_.index)

In [ ]:
cds_hist_.loc[date]

In [ ]:
date

In [ ]:
cds = get_cds(
    cds_name=cds_name,
    curve_name=curve_name,
    trade_date=trade_date
)

In [ ]:
100 * (1 + cds.fairUpfront())

In [ ]:
cds_ip

In [ ]:
cds.NPV()

In [ ]:
curve_hist_date_

In [ ]:
ch

In [ ]:
cds_ref_

In [ ]:
pd.read_csv('cip.csv')

In [ ]:
cd_ip.to_csv('cip.csv', index=False)

#### QuantLib

In [ ]:
# date = ql.Date(20, 6, 2022)
ql_trade_date = get_ql_date(trade_date)
ql_term_date = get_ql_date(get_term_date(trade_date))
ql_upfront_date = ql.WeekendsOnly().advance(ql_trade_date, 3*ql.Period(ql.Daily))
spot_date = ql.WeekendsOnly().advance(ql_trade_date, 2*ql.Period(ql.Daily))
period_parser = ql.PeriodParser()
ql.Settings.instance().setEvaluationDate(ql_trade_date)

In [ ]:
deposit_rate_helpers = [
    ql.DepositRateHelper(
        rate / 100,  # rate
        period_parser.parse(tenor),  # tenor                                      
        2,  # fixingDays 
        ql.WeekendsOnly(),
        ql.ModifiedFollowing,
        False, 
        ql.Actual360()
    ) for tenor, rate in deposit_data.items()
]

In [ ]:
isda_ibor = ql.IborIndex(
    'IsdaIbor',  # familyName
    3*ql.Period(ql.Monthly),  # tenor 
    2,  # settlementDays
    ql.USDCurrency(),  # currency
    ql.WeekendsOnly(),  # fixingCalendar
    ql.ModifiedFollowing, # convention
    False, # endOfMonth
    ql.Actual360(),  # dayCounter
)

swap_rate_helpers = [
    ql.SwapRateHelper(
        rate / 100,  # rate
        period_parser.parse(tenor),  # tenor
        ql.WeekendsOnly(),  # calendar
        ql.Semiannual,  # fixedFrequency
        ql.ModifiedFollowing,  # fixedConvention
        ql.Thirty360(),  # fixedDayCount
        isda_ibor,  # iborIndex
    ) for tenor, rate in swap_data.items()
]

isda_rate_helpers = [*deposit_rate_helpers, *swap_rate_helpers]
rate_curve = ql.PiecewiseLogLinearDiscount(ql_trade_date, isda_rate_helpers, ql.Actual365Fixed())
discount_curve = ql.YieldTermStructureHandle(rate_curve)

In [ ]:
cds_schedule = ql.Schedule(
    ql_trade_date, 
    ql_term_date,
    3*ql.Period(ql.Monthly),
    ql.WeekendsOnly(),
    ql.Following, 
    ql.Unadjusted,
    ql.DateGeneration.CDS,
    False
)

quoted_trade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,  # side
    10000000,  # nominal
    0,  # upfront
    cds_price['last_price'] / 10000,  # spread
    cds_schedule,  # Schedule
    ql.Following, 
    ql.Actual360(), 
    True, 
    True, 
    ql_trade_date,
    ql_upfront_date, 
    ql.FaceValueClaim(), 
    ql.Actual360(True)
)
# quoted_trade.setPricingEngine(engine)

h = quoted_trade.impliedHazardRate(
    0, 
    discount_curve, 
    ql.Actual365Fixed(),
    cds_ref['recovery'], 
    1e-10,
    ql.CreditDefaultSwap.ISDA
)

In [ ]:
fhr_curve = ql.FlatHazardRate(
    ql_trade_date, 
    ql.WeekendsOnly(),
    ql.QuoteHandle(ql.SimpleQuote(h)),
    ql.Actual365Fixed()
)
pd_curve = ql.DefaultProbabilityTermStructureHandle(fhr_curve)

engine = ql.IsdaCdsEngine(pd_curve, cds_ref['recovery'], discount_curve)

conventional_trade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,  # side
    10000000,  # notional
    0,  # upfront
    100 / 10000,  # spread
    cds_schedule,  # schedule
    ql.Following,  # paymentConvention
    ql.Actual360(),  # dayCounter
    True,  # settlesAccrual
    True,  # paysAtDefaultTime
    ql_trade_date,  # protectionStart
    ql_upfront_date,  # upfrontDate
    ql.FaceValueClaim(),  # Claim
    ql.Actual360(True),  # lastPeriodDayCounter
    True,  # rebatesAccrual
)
conventional_trade.setPricingEngine(engine)

In [ ]:
f"NPV: {conventional_trade.NPV()}, Price: {100 * (1+conventional_trade.fairUpfront())}"

In [ ]:
quoted_trade.coupons()

In [ ]:
ql.CashFlow.amount(conventional_trade.coupons())

In [ ]:
conventional_trade.couponLegNPV()

In [ ]:
conventional_trade.defaultLegNPV()

In [ ]:
conventional_trade.NPV()

In [ ]:
conventional_trade.accrualRebateNPV()

In [ ]:
1+conventional_trade.fairUpfront()

In [ ]:
quoted_trade.fairUpfront()


In [ ]:
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()

In [ ]:

termDate = ql.Date(20, 6, 2010),
             ql.Date(20, 6, 2011),
             ql.Date(20, 6, 2012),
             ql.Date(20, 6, 2016),
             ql.Date(20, 6, 2019)

spreads = [0.001, 0.1]
recoveries = [0.2, 0.4]

markitValues = [97798.29358,  # 0.001
                97776.11889,  # 0.001
                -914971.5977,  # 0.1
                -894985.6298,  # 0.1
                186921.3594,  # 0.001
                186839.8148,  # 0.001
                -1646623.672,  # 0.1
                -1579803.626,  # 0.1
                274298.9203,
                274122.4725,
                -2279730.93,
                -2147972.527,
                592420.2297,
                591571.2294,
                -3993550.206,
                -3545843.418,
                797501.1422,
                795915.9787,
                -4702034.688,
                -4042340.999]

tolerance = 1.0e-2

l = 0
distance = 0

data = []
upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
for termDate in termDates:
    for spread in spreads:
        for recovery in recoveries:
            cdsSchedule = ql.Schedule(
                trade_date, 
                termDate,
                3 * ql.Period(ql.Monthly),
                ql.WeekendsOnly(),
                ql.Following, 
                ql.Unadjusted,
                ql.DateGeneration.CDS,
                False
            )

            quotedTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 
                10000000, 
                0, 
                spread, 
                cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))

            h = quotedTrade.impliedHazardRate(
                0, 
                discountCurve, 
                ql.Actual365Fixed(),
                recovery, 
                1e-10,
                ql.CreditDefaultSwap.ISDA
            )

            probabilityCurve.linkTo(
                ql.FlatHazardRate(0, ql.WeekendsOnly(),
                                  ql.QuoteHandle(ql.SimpleQuote(h)),
                                  ql.Actual365Fixed()))

            engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
            conventionalTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 10000000, 0, 0.01, cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))
            conventionalTrade.setPricingEngine(engine)

            upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

            data.append(
                (termDate,
                 spread,
                 recovery,
                 h,
                 upfront,
                 markitValues[l],
                 abs(upfront - markitValues[l]),
                 abs(upfront - markitValues[l]) < tolerance)
            )
            distance = distance + abs(upfront - markitValues[l])

            l = l + 1

df = pd.DataFrame(data, columns=["Term date", "Spread", "Recovery",
                                 "Hazard rate", "Upfront", "Markit value", "Distance", "Within tolerance"])


df.style.format({'Spread': '{:.4%}', 'Hazard rate': '{:.2%}', 'Upfront': '{:.2f}',
                 'Markit value': '{:.2f}', 'Distance': '{:.6f}'})

In [ ]:
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()

termDates = [ql.Date(20, 6, 2010),
             ql.Date(20, 6, 2011),
             ql.Date(20, 6, 2012),
             ql.Date(20, 6, 2016),
             ql.Date(20, 6, 2019)]

spreads = [0.001, 0.1]
recoveries = [0.2, 0.4]

markitValues = [97798.29358,  # 0.001
                97776.11889,  # 0.001
                -914971.5977,  # 0.1
                -894985.6298,  # 0.1
                186921.3594,  # 0.001
                186839.8148,  # 0.001
                -1646623.672,  # 0.1
                -1579803.626,  # 0.1
                274298.9203,
                274122.4725,
                -2279730.93,
                -2147972.527,
                592420.2297,
                591571.2294,
                -3993550.206,
                -3545843.418,
                797501.1422,
                795915.9787,
                -4702034.688,
                -4042340.999]

tolerance = 1.0e-2

l = 0
distance = 0

data = []
upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
for termDate in termDates:
    for spread in spreads:
        for recovery in recoveries:
            cdsSchedule = ql.Schedule(
                trade_date, 
                termDate,
                3 * ql.Period(ql.Monthly),
                ql.WeekendsOnly(),
                ql.Following, 
                ql.Unadjusted,
                ql.DateGeneration.CDS,
                False
            )

            quotedTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 
                10000000, 
                0, 
                spread, 
                cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))

            h = quotedTrade.impliedHazardRate(
                0, 
                discountCurve, 
                ql.Actual365Fixed(),
                recovery, 
                1e-10,
                ql.CreditDefaultSwap.ISDA
            )

            probabilityCurve.linkTo(
                ql.FlatHazardRate(0, ql.WeekendsOnly(),
                                  ql.QuoteHandle(ql.SimpleQuote(h)),
                                  ql.Actual365Fixed()))

            engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
            conventionalTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 10000000, 0, 0.01, cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))
            conventionalTrade.setPricingEngine(engine)

            upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

            data.append(
                (termDate,
                 spread,
                 recovery,
                 h,
                 upfront,
                 markitValues[l],
                 abs(upfront - markitValues[l]),
                 abs(upfront - markitValues[l]) < tolerance)
            )
            distance = distance + abs(upfront - markitValues[l])

            l = l + 1

df = pd.DataFrame(data, columns=["Term date", "Spread", "Recovery",
                                 "Hazard rate", "Upfront", "Markit value", "Distance", "Within tolerance"])


df.style.format({'Spread': '{:.4%}', 'Hazard rate': '{:.2%}', 'Upfront': '{:.2f}',
                 'Markit value': '{:.2f}', 'Distance': '{:.6f}'})

In [ ]:
print('total distance:', distance)